In [1]:
from util import helpers
import wikipediaapi
import spacy
import re
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import opennre
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from operator import itemgetter

/home/hasan-sh/.virtualenvs/thesis/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# !pip install wikipedia-api spacy transformers sentence_transformers pytest

# !pip3 uninstall torch torchvision torchaudio #--extra-index-url https://download.pytorch.org/whl/cu113

# !python -m spacy download en_core_web_lg

In [2]:

entity = [{'type': 'uri', 'uri': 'http://yago-knowledge.org/resource/Borussia_Dortmund'}][0]['uri'].rsplit('/', 1)[-1]
noHints = [[{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '56903', 'uri': '56903', 'prefix_entity': "'56903'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'United_States', 'uri': 'http://yago-knowledge.org/resource/United_States', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:United\\_States'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '21762', 'uri': '21762', 'prefix_entity': "'21762'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'India', 'uri': 'http://yago-knowledge.org/resource/India', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:India'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '11454', 'uri': '11454', 'prefix_entity': "'11454'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'United_Kingdom', 'uri': 'http://yago-knowledge.org/resource/United_Kingdom', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:United\\_Kingdom'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '8714', 'uri': '8714', 'prefix_entity': "'8714'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'France', 'uri': 
'http://yago-knowledge.org/resource/France', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:France'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '7149', 'uri': '7149', 'prefix_entity': "'7149'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Germany', 'uri': 'http://yago-knowledge.org/resource/Germany', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Germany'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '4030', 'uri': '4030', 'prefix_entity': "'4030'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Canada', 'uri': 'http://yago-knowledge.org/resource/Canada', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Canada'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '3855', 'uri': '3855', 'prefix_entity': "'3855'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Japan', 'uri': 'http://yago-knowledge.org/resource/Japan', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Japan'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '3473', 'uri': '3473', 'prefix_entity': "'3473'"}, {'type': 'uri', 'value': 'type', 'uri': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'prefix': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'prefix_entity': 'x2:type'}, {'type': 'uri', 'value': 'Movie', 'uri': 'http://schema.org/Movie', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:Movie'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '2768', 'uri': '2768', 'prefix_entity': "'2768'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Australia', 'uri': 'http://yago-knowledge.org/resource/Australia', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Australia'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 
'literal', 'value': '1787', 'uri': '1787', 'prefix_entity': "'1787'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Soviet_Union', 'uri': 'http://yago-knowledge.org/resource/Soviet_Union', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Soviet\\_Union'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '183', 'uri': '183', 'prefix_entity': "'183'"}, {'type': 'uri', 'value': 'nationality', 'uri': 'http://schema.org/nationality', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:nationality'}, {'type': 'uri', 'value': 'India', 'uri': 'http://yago-knowledge.org/resource/India', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:India'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '155', 'uri': '155', 'prefix_entity': "'155'"}, {'type': 'uri', 'value': 'nationality', 'uri': 'http://schema.org/nationality', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:nationality'}, {'type': 'uri', 'value': 'Canada', 'uri': 'http://yago-knowledge.org/resource/Canada', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Canada'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '36', 'uri': '36', 'prefix_entity': "'36'"}, {'type': 'uri', 'value': 'hasOccupation', 'uri': 'http://schema.org/hasOccupation', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:hasOccupation'}, {'type': 'uri', 'value': 'Film_producer', 'uri': 'http://yago-knowledge.org/resource/Film_producer', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Film\\_producer'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '21', 'uri': '21', 'prefix_entity': "'21'"}, {'type': 'uri', 'value': 'hasOccupation', 'uri': 'http://schema.org/hasOccupation', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:hasOccupation'}, {'type': 'uri', 'value': 'Screenwriter', 'uri': 'http://yago-knowledge.org/resource/Screenwriter', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Screenwriter'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '20', 'uri': '20', 'prefix_entity': "'20'"}, {'type': 'uri', 'value': 'hasOccupation', 'uri': 'http://schema.org/hasOccupation', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:hasOccupation'}, {'type': 'uri', 'value': 'Film_director', 'uri': 'http://yago-knowledge.org/resource/Film_director', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Film\\_director'}]]

falseRelations = [[helpers.parseTriple(noHints[i])[1],helpers.parseTriple(noHints[i])[2]] for i in range(len(noHints))]

wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page('{0}'.format(entity))

model_BERT = pipeline("fill-mask")#,  model = "xlm-roberta-base")

model_embedding = SentenceTransformer('all-mpnet-base-v2')
model_embedding.cuda()
model_re = opennre.get_model('wiki80_cnn_softmax')
model_re.cuda()

nlp = spacy.load('en_core_web_lg')

doc = nlp(p_wiki.text)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
2022-04-18 07:50:10,109 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2022-04-18 07:50:23,065 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cuda
2022-04-18 07:50:26,335 - root - INFO - Initializing word embedding with word2vec.
2022-04-18 07:50:27,331 - wikipediaapi - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Borussia_Dortmund&explaintext=1&exsectionformat=wiki


In [3]:
def cosineSim(x, y):
    return cosine_similarity(model_embedding.encode(x, show_progress_bar=False).reshape(1, -1), model_embedding.encode(y, show_progress_bar=False).reshape(1, -1))

In [4]:
def extractRelations(sentence, subj, obj, falsePO, bert=False, reverse=False): 
    relations = []
    for sub_subj in subj:
        if sub_subj in sentence:
            head_s = sentence.index(sub_subj)
            head_e = head_s + len(sub_subj)
            tail_s = sentence.index(obj)
            tail_e = tail_s + len(obj)
            
            if bert:
                bestResults = model_BERT(f"{sentence} {sub_subj} <mask> {obj} ")
                print(bestResults, falsePO[0])
                
                i = np.argmax([cosineSim(x['token_str'], falsePO[0]) for x in bestResults[:2]])# if x['score'] > 0.3])
                bestResult = bestResults[i]
                print(bestResult)
                relation = (bestResult['token_str'], bestResult['score'])
            else:
                relation = model_re.infer({'text': sentence, 'h': {'pos': (head_s, head_e)}, 't': {'pos': (tail_s, tail_e)}})
                if reverse:
                    relation = model_re.infer({'text': sentence, 't': {'pos': (head_s, head_e)}, 'h': {'pos': (tail_s, tail_e)}})
                
            relations.append((sub_subj, relation, obj))
    return relations

In [5]:
def validateRelations(relations, falsePO):
    best = (0, relations[0])
    for relation in relations:
        sim = cosineSim(f'{relation[1][0]} {relation[2]}', ' '.join(falsePO))
        print(relation)
        if best[0] < sim:
            best = (sim, relation)
    return best

In [6]:
def compareClassifiers(sent, subj, obj, falsePO):
    # Calculate for NRE
    nreRelations = extractRelations(sent.text, subj, obj, falsePO) # get best 5 candidates, should we consider more?
    _, nreBestRelation = validateRelations(nreRelations, falsePO)
    print('Best NRE Relation Obtained: ', nreBestRelation)
    print('From NRE Relations', nreRelations, '\n')

    # Calculate for BERT
    bertRelations = extractRelations(sent.text, subj, obj, falsePO, bert=True) # get best 5 candidates, should we consider more?
    __, bertBestRelation = validateRelations(bertRelations, falsePO)
    print('Best BERT Relation Obtained: ', bertBestRelation)
    print('From BERT Relations', bertRelations, '\n')
    print(sent, obj)
    print('FALSE PO ',  falsePO)


    # Calculate the similarities and choose the best.
    p1 = cosineSim(nreBestRelation[1][0], falsePO[0])
    p2 = cosineSim(bertBestRelation[1][0], falsePO[0])

    falsePO = ' '.join(falsePO)
    print(falsePO)
    po1 = cosineSim(f'{nreBestRelation[1][0]} {nreBestRelation[2]}', falsePO)
    po2 = cosineSim(f'{bertBestRelation[1][0]} {bertBestRelation[2]}', falsePO)

    mean1 = np.mean([p1, po1])
    mean2 = np.mean([p2, po2])
    print(mean1, mean2)
    return ('nre', mean1, nreBestRelation) if mean1 > mean2 else ('bert', mean2, bertBestRelation)
    # pass

In [7]:
s = 'Swift was around 12 years old, computer repairman and local musician Ronnie Cremer taught her to play guitar.'
# s = 'Ballspielverein Borussia 09 e. V. Dortmund, commonly known as Borussia Dortmund (German pronunciation: [boˈʁʊsi̯aː ˈdɔɐ̯tmʊnt] (audio speaker iconlisten)),[2] BVB (pronounced [beːfaʊ̯ˈbeː] (audio speaker iconlisten)), or simply Dortmund (pronounced [ˈdɔʁtmʊnt] (audio speaker iconlisten)), is a German professional sports club based in Dortmund, North Rhine-Westphalia.'
# s = 'The Los Angeles Lakers is a good team. They are an American professional basketball team based in Los Angeles.'
# s = 'The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or simply America, is a country primarily located in North America.'
subject = ['Taylor Swift', 'Taylor', 'Swift'] # TODO: use PoS on the subject.
# subject = ['Dortmund', 'Borussia'] # TODO: use PoS on the subject.
# subject = ['Los Angeles Lakers', 'Los Angeles', 'Lakers', 'Dortmund'] # TODO: use PoS on the subject.
# subject = ['United States', 'States'] # TODO: use PoS on the subject.
bestO = 'guitar'
# bestO = 'North America'

falsePO = ['has occupation', 'Guitarist']

In [56]:
compareClassifiers(nlp(s), subject, bestO, falsePO)

('Swift', ('instrument', 0.8021246790885925), 'guitar')
Best NRE Relation Obtained:  ('Swift', ('instrument', 0.8021246790885925), 'guitar')
From NRE Relations [('Swift', ('instrument', 0.8021246790885925), 'guitar')] 

[{'score': 0.5662645697593689, 'token': 1974, 'token_str': ' plays', 'sequence': 'Swift was around 12 years old, computer repairman and local musician Ronnie Cremer taught her to play guitar. Swift plays guitar '}, {'score': 0.16256368160247803, 'token': 702, 'token_str': ' played', 'sequence': 'Swift was around 12 years old, computer repairman and local musician Ronnie Cremer taught her to play guitar. Swift played guitar '}, {'score': 0.07455544918775558, 'token': 13419, 'token_str': ' teaches', 'sequence': 'Swift was around 12 years old, computer repairman and local musician Ronnie Cremer taught her to play guitar. Swift teaches guitar '}, {'score': 0.0710236206650734, 'token': 816, 'token_str': ' playing', 'sequence': 'Swift was around 12 years old, computer repairm

('bert', 0.53417385, ('Swift', (' plays', 0.5662645697593689), 'guitar'))

In [14]:
cosineSim('musician', 'pianist'), cosineSim('guitar', 'guitarist')

(array([[0.7970922]], dtype=float32), array([[0.83837724]], dtype=float32))

In [ ]:
classifier(f"{s} Taylor Swift <mask> guitar ")

In [15]:
p1 = cosine_sim('instrument', 'had occupation')
p2 = cosine_sim('played', 'had occupation')

Batches: 100%|██████████| 1/1 [00:00<00:00, 45.99it/s]


In [16]:
po1 = cosine_sim('instrument guitar', 'had occupation Guitarist')
po2 = cosine_sim('played guitar', 'had occupation Guitarist')

Batches: 100%|██████████| 1/1 [00:00<00:00, 45.47it/s]


In [17]:
po1, po2

(array([[0.5554023]], dtype=float32), array([[0.7435489]], dtype=float32))

In [18]:
mean1 = np.mean([p1, po1])
mean2 = np.mean([p2, po2])

In [19]:
mean1, mean2

(0.36996472, 0.48026884)

In [ ]:
mean1, mean2

In [ ]:

classifier(f"{s} Taylor Swift <mask> guitar ")


In [ ]:
subject = ['Borussia Dortmund', 'Borussia', 'Dortmund']
falsePO = "country of origin Germany"
falseO = "Germany" 

candidates = []

# TODO: pick a more robust sentence splitter.
for sentence in doc.sents:
    for token in sentence:
        chosen = False
        if (token.text == "Borussia" or token.text == "Dortmund") and token.dep_ == "nsubj":
            # use coreference relations; whether Swift is actually taylor or whether her is referring to taylor as well!
            # candidates.append(sentence)
            chosen = True
            
        if chosen:
            similar_objs = [[cosine_sim(token.text, falseObject)[0], token] for token in sentence]
            similar_objs.sort(key=itemgetter(0), reverse=True)
            best_obj = similar_objs[0]    
        
            candidates.append([sentence, best_obj[0], best_obj[1]])

            
            
# for index, candidate in enumerate(candidates):
#     similar_objs = [[cosine_sim(token.text, falseObject)[0], token] for token in candidate]
    
#     similar_objs.sort(key=itemgetter(0), reverse=True)
#     best_obj = similar_objs[0]    
    
#     candidates[index] = [candidate, best_obj[0], best_obj[1]]

candidates.sort(key=itemgetter(1), reverse=True)
print(candidates[:5])

In [ ]:
for c in candidates[:5]:
    for sub_subject in subject:
        head_ 
        tail_start = c[0].index(c[2])
        tail_end = tail_start + len(c[2])
    
    

In [ ]:
candidates.sort(key=itemgetter(1), reverse=True)
for i in candidates[:10]:
    print(i, '\n')

In [ ]:
classifier(f"{s}. United States <mask> Los Angelos Lakers ")

In [ ]:

falseRelation = "Borussia Dortmund country of origin Germany"

candidates = []

# TODO: pick a more robust sentence splitter.
for sentence in doc.sents:
    for token in sentence:
        if (token.text == "Borussia" or token.text == "Dortmund") and token.dep_ == "nsubj":
            # use coreference relations; whether Swift is actually taylor or whether her is referring to taylor as well!
            candidates.append([cosine_sim(sentence.text, falseRelation), sentence])

candidates.sort(key=itemgetter(0), reverse=True)
print(candidates[:5])

In [ ]:
update = False
finalSentence = None
for candidate in candidates[:5]:
    # objects = [print(token, token.dep_) for token in candidate[1]]
    objects = [token for token in candidate[1] if token.dep_ == 'dobj' or token.dep_ == 'pobj']
    similarities = [(cosine_sim(o.text, 'Germany'), o) for o in objects]
    similarities.sort(reverse=True)
    print('All objects: ', objects, 'from', candidate)
    print('sim', similarities)
    obj = similarities[0]
    print('\n selected object', obj, '\n')
    if classifier(f"{candidate[1].text} Borussia Dortmund <mask> {obj[1].text}.")[0]["score"] >= .008: # TODO: consider a higher threshhold. WHat about ambiguity?
        result = classifier(f"{candidate[1].text} Borussia Dortmund <mask> {obj[1]}.")[0]
        pred = result['token_str']
        print('result ', result)
        if cosine_sim(f'{pred} {obj[1].text}', 'country of origin Germany') > 0.6: # TODO: change threshold!!
            update = True
            finalSentence = result['sequence']
            break
            
print(finalSentence, update)